In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# Alpha-Beta Pruning

Stroetmann, K. (2022) describes __alpha-beta pruning__ as an optimization of the minimax search algorithm. It aims to "cut off" (prune) any branches in the search tree that cannot affect the outcome, therefore not evaluating these branches and speeding up the search. The algorithm achieves this by keeping track of two additional parameters during the search, which are denoted as $α$ and $β$. $α$ can be considered the lower bound; it is the lowest possible value that the search could output at that time. In contrast, $β$ can be considered the upper bound; it is the highest possible value that the search could output at that point in time. $α$ and $β$ are regularly updated during the search to narrow down the possible range of values.  

Because alpha-beta pruning is an extension of minimax search, it uses a maximizing function and a minimizing function, called $alphaBetaMax$ and $alphaBetaMin$ respectively. $alphaBetaMax$ approximates minimax's $maxValue$ as follows:

1. $\alpha \leq \texttt{maxValue}(s) \leq \beta \;\rightarrow\;\texttt{alphaBetaMax}(s, \alpha, \beta) = \texttt{maxValue}(s)$
1. $\texttt{maxValue}(s) < \alpha \;\rightarrow\; \texttt{alphaBetaMax}(s, \alpha, \beta) \leq \alpha$
1. $\beta < \texttt{maxValue}(s) \;\rightarrow\; \beta \leq \texttt{alphaBetaMax}(s, \alpha, \beta)$

Similarly, $alphaBetaMin$ approximates minimax's $minValue$ as follows:

1. $\alpha \leq \texttt{minValue}(s) \leq \beta \;\rightarrow\;\texttt{alphaBetaMin}(s, \alpha, \beta) = \texttt{minValue}(s)$
1. $\texttt{minValue}(s) < \alpha \;\rightarrow\; \texttt{alphaBetaMin}(s, \alpha, \beta) \leq \alpha$
1. $\beta < \texttt{minValue}(s) \;\rightarrow\; \beta \leq \texttt{alphaBetaMin}(s, \alpha, \beta)$

Finally, if the upper bound `ub` and the lower bound $lb$ of $maxValue$ and $minValue$ are known, the following is true:

1. $\texttt{maxValue}(s) := \texttt{alphaBetaMax}(s, lb, ub)$
1. $\texttt{minValue}(s) := \texttt{alphaBetaMin}(s, lb, ub)$

Naturally, there are multiple implementations that meet this definition. This notebook presents one such implementation, which utilizes these properties to yield a significant performance increase compared to minimax search. Like the implementation of minimax search, this alpha-beta pruning implementation uses a limit $max_iterations$ denoting the maximum search depth, which is required to make the search realistic on current hardware.

---

### Dependencies

In [ ]:
!pip install python-chess
!pip install import_ipynb

     |████████████████████████████████| 147 kB 17.8 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=5eb58cf9ce15395c0220f1c3f828fff679f5938c8e3a8cc77c6a3325305b62a8
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import chess
import chess.gaviota
import random
import signal
import time
from typing import Union
from enum import Enum

import import_ipynb
import Util
from Globals import *

importing Jupyter notebook from Util.ipynb


importing Jupyter notebook from Globals.ipynb


#### chess.Board.get_best_move_alphabeta
Finds the best move to make based on the alpha-beta pruning algorithm. This algorithm works as follows:
- Iterate over all legal moves in the current position.
- For each move, find the best possible score after making this move.
    - This is done by calling the alpha-beta pruning function recursively: increasing the current iteration by 1 and switching turns.
    - If a move yields a lower score than the previously examined move in the hierarchy when maximizing, or a higher score when minimizing, it cannot be the optimal move. Therefore, such moves are not evaluated further.
- Find the maximum (if it's the AI's turn) or the minimum (if it's other player's/AI's turn) score of all legal moves, alongside the move that was able to reach this optimized state. This is the move that the algorithm recommends.

Optionally, the algorithm uses memoization, which is a type of caching. This works by mapping a board state, plus iteration details, to the corresponding score and move and storing it in a cache dictionary. If, during a later iteration, the same board state is reached on the same iteration, these values can be read from the cache. Alpha-beta pruning additionally requires one to account for the values of alpha and beta. This is further detailed later in this notebook.

This function combines the abovementioned $alphaBetaMax$ and $alphaBetaMin$ because of their functional overlap. If $ai\_turn = True$, then the function implements $alphaBetaMax$; otherwise, it implements $alphaBetaMin$.

__This function is implemented recursively, but indirectly: it calls other functions that can in turn call this function again.__

###### <u>__Arguments__</u>
``cache (dict):``  
A cache dictionary if memoization is desired, or None if memoization should be disabled.

``use_heuristic (bool):``  
Whether or not the heuristic for evaluating the chess board should be used. Chess problems don't need this heuristic.  

``alpha (int):``  
The "alpha" value in the current iteration. On the first iteration, this value should be strongly negative.  

``beta (int):``  
The "beta" value in the current iteration. On the first iteration, this value should be strongly positive.  

``ai_turn (bool):``  
Whether or not the current turn is of the AI that started the search. If this is the case, the score should be maximized. Otherwise, the score should be minimized.  

``color (chess.Color):``  
The color of the player whose turn it currently is.  

``endgame_tablebase (Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase]):``  
The endgame tablebase attached to the game, which serves as a shortcut for ideal moves in the endgame.  

``iteration (int):``  
The depth of the search (amount of moves currently looking ahead).  

``max_iterations (int): ``  
The maximum depth of the search.  

``using_progressive_deepening (bool):``   
Whether or not the search is part of a progressive deepening search.  

``last_eval_score (int):``  
The score provided by the previous evaluation in the search.  

###### __<u>Returns _(int, chess.Move, bool)_</u>__
- The board score after making the recommended best move.
- The recommended best move to make.
- Whether or not the endgame library was used to find the move.

###### __<u>Side effects</u>__
- If a cache dictionary is provided, new values may be added to this dictionary.
- If the search is interrupted, the board may be in a different state than when the search started.

In [ ]:
def get_best_move_alphabeta(
    self,
    cache: dict,
    use_heuristic: bool,
    alpha: int,
    beta: int,
    ai_turn: bool,
    color: chess.Color,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase,
            chess.gaviota.PythonTablebase],
    iteration: int,
    max_iterations: int,
    using_progressive_deepening: bool,
    last_eval_score: int
) -> (int, chess.Move, bool):

    state = self.get_state_repr()
    original_color = color if ai_turn else not color
    
    # Read cached result if available
    if cache is not None:                
        alpha, beta, cached_entry = self.get_alphabeta_cache(
                cache, iteration, max_iterations, state, alpha, beta)
        if cached_entry is not None: return cached_entry

    # Get result if the search has finished (if max iterations has been reached,
    # or if the game has ended)
    result_score = self.get_search_result_if_finished(original_color, iteration,
            max_iterations, last_eval_score)
    if result_score is not None:
        return result_score, None, False

    # Get result if the search has not finished
    result = self.value_alphabeta(cache, use_heuristic, alpha, beta, ai_turn,
            color, endgame_tablebase, iteration, max_iterations,
            using_progressive_deepening, last_eval_score)

    # Store result in cache
    if cache is not None:
        self.store_alphabeta_cache(cache, iteration, max_iterations,
                state, alpha, beta, result)
        
    return result

chess.Board.get_best_move_alphabeta = get_best_move_alphabeta
del get_best_move_alphabeta

#### chess.Board.value_alphabeta
Finds the best move at the current node in the alpha-beta pruning search. This assumes that the search has not ended at this node, and that the result at the current board state and iteration is not available in the cache.

__This function is implemented recursively, but indirectly: it calls other functions that can in turn call this function again.__

###### __<u>Arguments</u>__
The arguments are the same as in ``chess.Board.get_best_move_alphabeta``. To save space, these will not be repeated here.

###### __<u>Returns _(int, chess.Move, bool)_</u>__
- The board score after making the recommended best move.
- The recommended best move to make.
- Whether or not the endgame library was used to find the move.

###### __<u>Side effects</u>__
- If the search is interrupted, the board may be in a different state than when the search started.

In [ ]:
def value_alphabeta(
    self,
    cache: dict,
    use_heuristic: bool,
    alpha: int,
    beta: int,
    ai_turn: bool,
    color: chess.Color,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase,
            chess.gaviota.PythonTablebase],
    iteration: int,
    max_iterations: int,
    using_progressive_deepening: bool,
    last_eval_score: int
) -> (int, chess.Move, bool):

    sve_result = self.single_value_extension_alphabeta(cache, use_heuristic,
            alpha, beta, ai_turn, color, endgame_tablebase, iteration,
            max_iterations, using_progressive_deepening, last_eval_score)
    if sve_result is not None:
        return sve_result

    available_moves = self.get_sorted_moves(cache, ai_turn,
                    iteration, max_iterations) \
            if using_progressive_deepening \
            else self.legal_moves

    return self.evaluate_moves_alphabeta(available_moves, cache, use_heuristic,
            alpha, beta, ai_turn, color, endgame_tablebase, iteration,
            max_iterations, using_progressive_deepening, last_eval_score)

chess.Board.value_alphabeta = value_alphabeta
del value_alphabeta

#### chess.Board.single_value_extension_alphabeta
Implements single value extension in the alpha-beta pruning search. This is an optimization for when only one legal move can be made. In this case, this move is evaluated without increasing the iteration counter, since searching through only one node adds a neglegible amount of time.

__This function is implemented recursively, but indirectly: it calls other functions that can in turn call this function again.__

###### __<u>Arguments</u>__
The arguments are the same as in `chess.Board.get_best_move_alphabeta`. To save space, these will not be repeated here.

###### __<u>Returns _(int, chess.Move, bool)_</u>__
- The board score after making the recommended best move.
- The recommended best move to make.
- Whether or not the endgame library was used to find the move.

###### __<u>Side effects</u>__
- If the search is interrupted, the board may be in a different state than when the search started.

In [ ]:
def single_value_extension_alphabeta(
    self,
    cache: dict,
    use_heuristic: bool,
    alpha: int,
    beta: int,
    ai_turn: bool,
    color: chess.Color,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase,
            chess.gaviota.PythonTablebase],
    iteration: int,
    max_iterations: int,
    using_progressive_deepening: bool,
    last_eval_score: int
) -> (int, chess.Move, bool):

    if len(list(self.legal_moves)) != 1:
        return None

    move = list(self.legal_moves)[0]
    self.push(move)
    score, _, used_endgame = self.get_best_move_alphabeta(
            cache, use_heuristic, alpha, beta, not ai_turn, not color,
            endgame_tablebase, iteration, max_iterations,
            using_progressive_deepening, last_eval_score)
    self.pop()
    return score, move, used_endgame

chess.Board.single_value_extension_alphabeta = single_value_extension_alphabeta
del single_value_extension_alphabeta

#### chess.Board.evaluate_moves_alphabeta
Iterates through the available legal moves and searches their move trees using alpha-beta pruning. Once all moves are evaluated this way, the best move and its associated properties are returned.

__This function is implemented recursively, but indirectly: it calls other functions that can in turn call this function again.__

###### __<u>Arguments</u>__
``available_moves (list<chess.Move>):``  
A list of available legal moves.  

The other arguments are the same as in `chess.Board.get_best_move_alphabeta`. To save space, these will not be repeated here.

###### __<u>Returns _(int, chess.Move, bool)_</u>__
- The board score after making the recommended best move.
- The recommended best move to make.
- Whether or not the endgame library was used to find the move.

###### __<u>Side effects</u>__
- If the search is interrupted, the board may be in a different state than when the search started.

In [ ]:
def evaluate_moves_alphabeta(
    self,
    available_moves: list,
    cache: dict,
    use_heuristic: bool,
    alpha: int,
    beta: int,
    ai_turn: bool,
    color: chess.Color,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase,
            chess.gaviota.PythonTablebase],
    iteration: int,
    max_iterations: int,
    using_progressive_deepening: bool,
    last_eval_score: int
)  -> (int, chess.Move, bool):

    best_score = Globals.MAX_EVALUATION_SCORE * (-1 if ai_turn else 1)
    best_move = None
    best_move_used_endgame = False

    for move in available_moves:
        eval_score, used_endgame_anywhere = self.evaluate_move(
                use_heuristic, color, not ai_turn, last_eval_score,
                iteration, move, endgame_tablebase)

        is_capture = self.is_capture(move)
        self.push(move)            
        
        if self.is_game_over():
            score_after_move = eval_score
        else:
            # Quiescence search - don't end on a move that isn't "quiet",
            # i.e. if a capture took place. This makes sure that potential
            # re-captures are taken into account.
            if iteration == max_iterations - 1 and is_capture:
                new_iteration = iteration
            else:
                new_iteration = iteration + 1

            score_after_move, _, used_endgame = self.get_best_move_alphabeta(
                    cache, use_heuristic, alpha, beta, not ai_turn, not color,
                    endgame_tablebase, new_iteration, max_iterations,
                    using_progressive_deepening, eval_score)
            used_endgame_anywhere = used_endgame_anywhere or used_endgame
                
        self.pop()          
                                                   
        if (ai_turn and score_after_move > best_score) \
                or (not ai_turn and score_after_move < best_score):
            best_score = score_after_move
            best_move = move
            best_move_used_endgame = used_endgame_anywhere

        if ai_turn:
            if best_score >= beta:
                return best_score, best_move, best_move_used_endgame
            if best_score > alpha:
                alpha = best_score
        else:
            if best_score <= alpha:
                return best_score, best_move, best_move_used_endgame
            if best_score < beta:
                beta = best_score
                
    return best_score, best_move, best_move_used_endgame  

chess.Board.evaluate_moves_alphabeta = evaluate_moves_alphabeta
del evaluate_moves_alphabeta  

---

### Memoization For Alpha-Beta Pruning
Because the same state on the same iteration may yield a different score if the values for α and β are different, it is required to include α and β in the cache. While it would be correct to simply store these values in the cache and only use the cached values if α and β are equal to their cached counterparts, this is unoptimal. Because chess evaluation scores can vary so greatly, it is unlikely for α and β to be exactly equal to their cached values, meaning that the cache can rarely be used. As described earlier, α and β are bounds for evaluation scores. This property can be used to improve the percentage of cache hits while still producing a correct score, see Stroetmann, K. (2022).

Cached values are tuples $(flag, v)$. $v$ is the cached evaluation score. $flag$ can be $≤$, $=$, or $≥$. Given a dictionary $cache$, this tuple represents the following:
* $\texttt{cache[s]} = ('=', v) \rightarrow \texttt{value(s)} = v$.
* $\texttt{cache[s]} = ('≤', v) \rightarrow \texttt{value(s)} \leq v$.
* $\texttt{cache[s]} = ('≥', v) \rightarrow \texttt{value(s)} \geq v$.

We define a function $evaluate$ that takes a state $s$, a function $f$ (either $alphaBetaMax$ or $alphaBetaMin$), an alpha value $α$ and a beta value $β$. This function returns the score of board state $s$ given optimal play. Given this function, the following is true:

1. $\texttt{cache[s]} = ('=', v) \rightarrow \texttt{evaluate}(\texttt{s}, f, \alpha, \beta) = v.$
1. $\texttt{cache[s]} = ('≤', v) \wedge v \leq \alpha \rightarrow \texttt{evaluate}(\texttt{s}, f, \alpha, \beta) = v.$
1. $\texttt{cache[s]} = ('≤', v) \wedge \alpha < v < \beta \rightarrow  \texttt{evaluate}(\texttt{s}, f, \alpha, \beta) = f(\texttt{s}, \alpha, v).$
1. $\texttt{cache[s]} = ('≤', v) \wedge \beta \leq v \rightarrow  \texttt{evaluate}(\texttt{s}, f, \alpha, \beta) = f(\texttt{s}, \alpha, \beta).$
1. $\texttt{cache[s]} = ('≥', v) \wedge \beta \leq v \rightarrow  \texttt{evaluate}(\texttt{s}, f, \alpha, \beta) = v.$
1. $\texttt{cache[s]} = ('≥', v) \wedge \alpha < v < \beta \rightarrow  \texttt{evaluate}(\texttt{s}, f, \alpha, \beta) = f(\texttt{s}, v, \beta).$
1. $\texttt{cache[s]} = ('≥', v) \wedge v \leq \alpha \rightarrow \texttt{evaluate}(\texttt{s}, f, \alpha, \beta) = f(\texttt{s}, \alpha, \beta).$

---

### Comparisons Class
An enum that contains values for the following comparisons:
- Equal to
- Less than or equal to
- Greater than or equal to

These values are used for alpha-beta pruning memoization and replace the single-character strings mentioned in the theory above, as integer constants are more efficient.

In [ ]:
class Comparisons(Enum):
    EQUAL = 0
    LTE = 1 # Less than or equal to
    GTE = 2 # Greater than or equal to

#### chess.Board.get_alphabeta_cache
Retrieves a cached value in an alpha-beta pruning search, based on the 7 equations listed in the theory above.

###### __<u>Arguments</u>__
``cache (dict):``  
The cache dictionary to read.  

``iteration (int):``  
The depth of the current node that's being evaluated.  

``max_iterations (int):``  
The maximum search depth used in the alpha-beta pruning search.  

``state (str):``  
A representation of the current chess board state.  

``alpha (int):``  
The "alpha" value in the current iteration.  

``beta (int):``  
The "beta" value in the current iteration.

###### __<u>Returns _(int, int, (int, chess.Move, bool))_</u>__
- The new "alpha" value.
- The new "beta" value.
- The move score, move and whether or not the endgame tablebase was used; or None if no value could be read from the cache.

In [ ]:
def get_alphabeta_cache(
    self,
    cache: dict,
    iteration: int,
    max_iterations: int,
    state: str,
    alpha: int,
    beta: int
) -> (int, int, (int, chess.Move, bool)):

    # Note: if a result is returned (i.e. not None) then alpha and beta
    # don't matter anymore

    key = (iteration, max_iterations, state)
    if key not in cache:
        return alpha, beta, None

    flag, result = cache[key]
    score, _, _ = result
    if flag == Comparisons.EQUAL:
        return alpha, beta, result
    if flag == Comparisons.LTE:
        if score <= alpha:
            return alpha, beta, result
        if score < beta: # alpha < score < beta
            return alpha, score, None
    if flag == Comparisons.GTE:
        if beta <= score:
            return alpha, beta, result
        if alpha < score: # alpha < score < beta
            return score, beta, None

    # Invalid flag or no optimization possible
    return alpha, beta, None

chess.Board.get_alphabeta_cache = get_alphabeta_cache
del get_alphabeta_cache

#### chess.Board.store_alphabeta_cache
Caches a board score in an alpha-beta pruning search, based on the representation of the cached tuples as described in the theory above.

###### __<u>Arguments</u>__
``cache (dict):``  
The cache dictionary to write to.  

``iteration (int):``  
The depth of the current node that's being evaluated.  

``max_iterations (int):``  
The maximum search depth used in the alpha-beta pruning search.  

``state (str):``  
A representation of the current chess board state.  

``alpha (int):``  
The "alpha" value in the current iteration.  

``beta (int):``  
The "beta" value in the current iteration.  

``result (tuple(int, chess.Move, bool)):``  
The move score, move and whether or not the endgame tablebase was used.

In [ ]:
def store_alphabeta_cache(
    self,
    cache: dict,
    iteration: int,
    max_iterations: int,
    state: str,
    alpha: int,
    beta: int,
    result: tuple
) -> None:

    key = (iteration, max_iterations, state)
    score, _, _ = result

    if score <= alpha:
        cache[key] = (Comparisons.LTE, result)
    elif score < beta: # alpha < score < beta
        cache[key] = (Comparisons.EQUAL, result)
    else: # beta <= score
        cache[key] = (Comparisons.GTE, result)

chess.Board.store_alphabeta_cache = store_alphabeta_cache
del store_alphabeta_cache

#### chess.Board.alphabeta_value_cache
Returns a cached move score if available, or a score of 0 if not. Can be used to sort moves by score.

###### __<u>Arguments</u>__
``cache (dict):``  
The cache dictionary to read.  

``move (chess.Move):``  
The move whose potentially cached score should be found.  

``iteration (int):``  
The current depth in the alpha-beta pruning search.

``max_iterations (int):``  
The maximum search depth used in the alpha-beta pruning search.

###### __<u>Returns _(int)_</u>__
The cached move score, or 0 if unavailable.

In [ ]:
def alphabeta_value_cache(
    self,
    cache: dict,
    move: chess.Move,
    iteration: int,
    max_iterations: int
) -> int:
    if max_iterations < 1:
        return 0
        
    self.push(move)
    state = self.get_state_repr()
    self.pop()
    
    _, result = cache.get((0, max_iterations, state), ('=', (0, None, False)))
    score, _, _ = result
    return score

chess.Board.alphabeta_value_cache = alphabeta_value_cache
del alphabeta_value_cache

#### make_move_alphabeta
Finds the best possible move according to the alpha-beta pruning algorithm and pushes it onto the move stack.

###### <b><u>Arguments</u></b>
``board (chess.Board):``  
The board to push the move to.  

``color (chess.Color):``  
The color of the player that makes the move.  

``search_depth (int):``  
The iteration depth of the alpha-beta search.  

``endgame_tablebase (Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase]):``  
The endgame tablebase attached to the game, which serves as a shortcut for ideal moves in the endgame.  

``use_heuristic (bool):``  
Whether or not the heuristic for evaluating the chess board should be used. Chess problems don't need this heuristic.

###### __<u>Returns _(int, chess.Move, bool)_</u>__
- The evaluated score of the best possible move.
- The best possible move that was found.
- Whether or not the endgame library was used to find the move.

###### __<u>Side effects</u>__
- The best possible move is pushed to the move stack of the board.

In [ ]:
def make_move_alphabeta(
    board: chess.Board,
    color: chess.Color,
    search_depth: int,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase,
            chess.gaviota.PythonTablebase],
    use_heuristic: bool,
) -> (int, chess.Move, bool):

    score, move, used_endgame = board.get_best_move_alphabeta(
        cache={}, 
        use_heuristic=use_heuristic,
        alpha=-Globals.MAX_EVALUATION_SCORE, 
        beta=Globals.MAX_EVALUATION_SCORE, 
        ai_turn=True, 
        color=color,
        endgame_tablebase=endgame_tablebase,
        iteration=0, 
        max_iterations=search_depth,
        using_progressive_deepening=False,
        last_eval_score=0 # The starting board score doesn't matter,
                          # it's evaluated by score difference
    )
    
    board.push(move)
    return score, move, used_endgame

---

### Progressive deepening

__Progressive deepening__, sometimes known as iterative deepening, is the idea of gradually inreasing the maximum search depth up to a certain number. In the context of games, this method can be used if information from a previous search is useful for a current search, even if that previous search used a lower search depth.

Stroetmann, K. (2022) explains that this is the case in alpha-beta pruning. In alpha-beta pruning, it is advantageous to evaluate the best moves first. These moves change the values of α and β the most significantly. Therefore, more branches of the search can be pruned, decreasing the amount of nodes to evaluate and speeding up the search. If the move scores of previous searches in a progressive deepening implementation are cached, these values can be used to estimate the effectiveness of each move, allowing the current search to sort the move list using these scores. This creates a best-first ordered list of moves, allowing the aforementioned optimization of alpha-beta pruning.

Because the amount of nodes that a search needs to evaluate grows exponentially with its search depth, searching the tree with a lower search depth takes comparatively little time. Generally, the time saved from sorting moves outweighs the time spent on performing these extra searches.

---

#### make_move_alphabeta_progressive_deepening
Finds the best possible move according to the alpha-beta pruning algorithm and pushes it onto the move stack. Utilizes progressive deepening by iteratively increasing the maximum search depth.

###### __<u>Arguments</u>__
``board (chess.Board):``  
The board to push the move to.  

``color (chess.Color):``  
The color of the player that makes the move.  

``search_depth (int):``  
The iteration depth of the alpha-beta search.  

``endgame_tablebase (Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase]):``  
The endgame tablebase attached to the game, which serves as a shortcut for ideal moves in the endgame.  

``use_heuristic (bool):``  
Whether or not the heuristic for evaluating the chess board should be used. Chess problems don't need this heuristic.

###### __<u>Returns _(int, chess.Move, bool)_</u>__
- The evaluated score of the best possible move.
- The best possible move that was found.
- Whether or not the endgame library was used to find the move.

###### __<u>Side effects</u>__
- The best possible move is pushed to the move stack of the board.

In [ ]:
def make_move_alphabeta_progressive_deepening(
    board: chess.Board,
    color: chess.Color,
    search_depth: int,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase,
            chess.gaviota.PythonTablebase],
    use_heuristic: bool,
) -> (int, chess.Move, bool):

    cache = {}
    score, move, used_endgame = 0, None, False

    for limit in range(1, search_depth + 1):
        score, move, used_endgame = board.get_best_move_alphabeta(
            cache=cache,
            use_heuristic=use_heuristic,
            alpha=-Globals.MAX_EVALUATION_SCORE,
            beta=Globals.MAX_EVALUATION_SCORE,
            ai_turn=True,
            color=color,
            endgame_tablebase=endgame_tablebase,
            iteration=0,
            max_iterations=limit,
            using_progressive_deepening=True,
            last_eval_score=0 # The starting board score doesn't matter,
                              # it's evaluated by score difference
        )

        # If a checkmate in favor of the AI has been found, there's no
        # reason to search any further
        if score >= Globals.EVALUATION_SCORE_CHECKMATE - limit:
            board.push(move)
            return score, move, used_endgame

    board.push(move)
    return score, move, used_endgame

#### chess.Board.get_sorted_moves
Returns a list of legal moves, sorted by move score, descending. This lets alpha-beta pruning search through the best move trees first.

###### __<u>Arguments</u>__
``cache (dict):``  
The cache dictionary to read.  

``ai_turn (bool):``  
Whether or not it's the turn of the AI that started the search.  

``iteration (int):``  
The current depth in the alpha-beta pruning search.

``max_iterations (int):``  
The maximum search depth used in the alpha-beta pruning search.

###### __<u>Returns _(list)_</u>__
The sorted list of legal moves.

In [ ]:
def get_sorted_moves(
    self,
    cache: dict,
    ai_turn: bool,
    iteration: int,
    max_iterations: int
) -> list:
    return sorted(
        self.legal_moves,
        reverse=ai_turn,
        key=lambda move: self.alphabeta_value_cache(cache, move,
                iteration, max_iterations - 2)
    )

chess.Board.get_sorted_moves = get_sorted_moves
del get_sorted_moves

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>